<a href="https://colab.research.google.com/github/dnguyend/MiscCollection/blob/main/EmbeddedDynamics/EmbeddedDynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\DeclareMathOperator{\ad}{ad}$
$\newcommand{\cI}{\mathcal{I}}$
$\DeclareMathOperator{\adI}{ad_{\cI}}$
$\newcommand{\mrGamma}{\mathring{\Gamma}}$
# Comparing with Euler-Arnold's type formula for curvature of $SE(n)$ with left-invariant metric
* Only the $SO(n)$ component contributes to the metric.
* Euler-Arnold: for antisymmetric matrices $A, B, C$ define the $\cI$-bracket
$$[A, B]_{\cI} := [A, B] + \cI^{-1}[A, \cI(B)] + \cI^{-1}[B, \cI(A)]$$
* Curvature using Euler-Arnold:
$$R = -\frac{1}{2}[[A, B], C]_{\cI} + \frac{1}{4}[A, [B,C]_{\cI}]_{\cI} -
\frac{1}{4}[B[A,C]_{\cI}]_{\cI}
$$
Set $\adI(A, B) := \cI^{-1}[A,\cI(B)]$
* $[A, B]_{\cI} = [A, B] + \adI(A, B) +\adI(B, A)$

For tangent vectors $A, B, C$ to $SO(n)$ at $I_n$:
* $\Gamma = -(AB)_{sym} +AB_{sym} +\frac{1}{2}(\adI(B, A) + \adI(A, B))$
* $\mrGamma = \frac{1}{2}(\adI(B, A) + \adI(A, B))$
* $D_A\mrGamma(B, C) = \frac{1}{2}A\{\adI(B, C)+ \adI(C, B)\} +
\frac{1}{2}\cI^{-1}\{[-AB, \cI(C)] + [B,-\cI(AC)] + [-AC, \cI(B)] +[C, -\cI(AB)]\}$
* Curvature using the embedded method:
$$R^{Emb}_{AB}C = D_A\mrGamma(B, C) - D_B\mrGamma(A, C)
+ \Gamma(A, \Gamma(B, C)) - \Gamma(B, \Gamma(A, C))$$

We verify both curvature formulas agree numerically below. Presumably we can verify they agree algebraically for another confirmation - but both formulas are long when expanded.

The sign convention for curvature in the reference below is opposite of the sign convention used here.

[1] D. Nguyen, Curvatures of Stiefel manifolds with deformation metrics, Journal of Lie Theory 32 (2022) 563–600.

In [1]:
import numpy as np


def sym(A):
  return 0.5*(A+A.T)

def asym(A):
  return 0.5*(A-A.T)

def genImat(n):
  Imat = np.abs(np.random.randn(n , n))
  Imat = sym(Imat)
  Imat[np.diag_indices(n)] = 1
  return Imat

def Iop(Imat, A):
  return A*Imat

def Iinv(Imat, A):
  return A/Imat

def Lie(A, B):
  return A@B - B@A

def adI(Imat, A, B):
  return Iinv(Imat, Lie(A, Iop(Imat, B)))

def LieI(Imat, A, B):
  return Lie(A, B) + adI(Imat, A, B) + adI(Imat, B, A)

def mrGamma(Imat, A, B):
  return 0.5*(adI(Imat, A, B) + adI(Imat, B, A))

def Gamma(Imat, A, B):
  return -sym(A@B) + A@sym(B) + 0.5*(adI(Imat, A, B) + adI(Imat, B, A))

def DmrGamma(Imat, A, B, C):
  return 0.5*A@adI(Imat, B, C) \
    + 0.5*A@adI(Imat, C, B) \
    + 0.5*adI(Imat, -A@B, C) \
    + 0.5*adI(Imat, B, -A@C) \
    + 0.5*adI(Imat, -A@C, B) \
    + 0.5*adI(Imat, C, -A@B)

def randvec(n):
  return asym(np.random.randn(n, n))

def CurvI(Imat, A, B, C):
  return -0.5*LieI(Imat, Lie(A, B), C) \
    + 0.25*LieI(Imat, A, LieI(Imat, B, C)) \
    - 0.25*LieI(Imat, B, LieI(Imat, A, C))

def CurvEmbedded(Imat, A, B, C):
  return DmrGamma(Imat, A, B, C) \
    - DmrGamma(Imat, B, A, C) \
    + Gamma(Imat, A, Gamma(Imat, B, C)) \
    - Gamma(Imat, B, Gamma(Imat, A, C))


## Test for n=2 to 10, for 10 scenarios each

In [2]:
def cz(mat):
  return np.max(np.abs(mat))
np.random.seed(0)
for n in range(2, 10):
  for i in range(10):
    A = randvec(n)
    B = randvec(n)
    C = randvec(n)

    Imat = genImat(n)

    c1 = CurvI(Imat, A, B, C)
    c2 = CurvEmbedded(Imat, A, B, C)
    display(cz(c1 - c2))




0.0

0.0

1.3877787807814457e-17

0.0

2.7755575615628914e-17

...

1.5987211554602254e-14

1.2434497875801753e-14

6.217248937900877e-15

9.325873406851315e-15

1.4210854715202004e-14

$\newcommand{\fk}{\mathfrak{k}}$
$\newcommand{\fp}{\mathfrak{p}}$
# Comparing Kobayashi-Nomizu II, proposition 3.4 and our formula for curvature, section 7.2
* Kobayashi Nomizu, 
$$KN = \frac{1}{4}[A[B,C]_{\fp}]_{\fp}-\frac{1}{4}[B[A,C]_{\fp}]_{\fp}
-\frac{1}{2}[[A,B]_{\fp}C]_{\fp} -[[A,B]_{\fk}C]\\
$$
* Since $\fk, \fp]\subset\fp$: $[[A,B]_{\fk}C] = [[A,B]_{\fk}C]_{\fp}$
$$KN = \frac{1}{4}[A[B,C]_{\fp}]_{\fp}-\frac{1}{4}[B[A,C]_{\fp}]_{\fp}
-\frac{1}{2}[[A,B]_{\fp}C]_{\fp} -\frac{1}{2}[[A,B]_{\fk}C]_{\fp}
-\frac{1}{2}[[A,B]_{\fk}C]_{\fp}\\
= \frac{1}{4}[A[B,C]]_{\fp} - \frac{1}{4}[A[B,C]_{\fk}]_{\fp}
-\frac{1}{4}[B[A,C]]_{\fp} +\frac{1}{4}[B[A,C]_{\fk}]_{\fp}
-\frac{1}{2}[[A,B]C]_{\fp} 
-\frac{1}{2}[[A,B]_{\fk}C]_{\fp}
$$
* Using the Jacobi's identity and $\fk, \fp]\subset\fp$ again
$$KN = - \frac{1}{4}[A[B,C]_{\fk}]
 +\frac{1}{4}[B[A,C]_{\fk}]
-\frac{1}{4}[[A,B]C]_{\fp} 
-\frac{1}{2}[[A,B]_{\fk}C]
=\frac{1}{4}\{[C[A,B]]_{\fp} -2[[A,B]_{\fk}C]
+ \frac{1}{4}[[B,C]_{\fk}A]
+\frac{1}{4}[[C,A]_{\fk}B]
\}
$$
QED

[2] Kobayashi S., Nomizu, K. Foundations of Differential Geometry
ISBN:‎ 978-0471157328, Wiley-Interscience; 1st edition (1996)
